In [1]:
from networkx.generators.random_graphs import random_regular_graph
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy.special import expit
from scipy.optimize import minimize
from Unit import unit
import pickle
import copy

In [2]:
#Parameters for generating data
nodeList = {}

Cscale = [[1.5, 3], [6, 2], [.8, .8]]
Uscale = [[2.3, 1.1], [.9, 1.1], [2, 2]]
tauA = {'intcp': -1, 'C0': .5, 'C1': .2, 'C2': .25, 'U0': .3, 'U1': -.2, 'U2': .25}
tauM = {'intcp': -1, 'C0': -.3, 'C1': .4, 'C2': .1, 'A': 1, 'nborA': -.5, 'nborM': -1.5} #nborM 1.5
tauY = {'intcp': -.3, 'nborA' : -1, 'M': 3, 'C0': -.2, 'C1': .2, 'C2': -.05, 'U0': .1, 
        'U1': -.2, 'U2': .25}


In [3]:
#Create a networkx graph and store it to disk (call either this cell or the next cell BUT NOT BOTH)
graph = random_regular_graph(3, 2000, seed=12345)
with open('./2000_3/graph2000_3.pkl', 'wb') as fname:
    pickle.dump(graph, fname)

In [3]:
# #Load a networkx graph from disk (call either this cell or the one preceding it BUT NOT BOTH)
# with open('./800_3/graph800_3.pkl', 'rb') as fname:
#     graph = pickle.load(fname)

In [4]:
#Function for sampling from the joint distribution p(M1, M2, ...)
#Inputs: numIter - number of iterations to run the sampler, M - a dummy initialization matrix,
#        tauM - parameters for M, nodeList - dictionary representation of the nodes in the graph (elements are Units; see Unit.py)
#Output: M - a single sample from the joint p(M1, M2, ...)
def Gibbs(numIter, M, tauM, nodeList):    
        for k in range(1, numIter+1):
            for i in range(len(nodeList)):
                sumM = tauM['intcp'] + (tauM['C0'] * nodeList[i].C[0]) + (tauM['C1'] * nodeList[i].C[1])
                sumM += (tauM['C2'] * nodeList[i].C[2])
                sumM += (tauM['A'] * nodeList[i].A)

                
                for nbor in nodeList[i].adj:
                    sumM += (tauM['nborA'] * nodeList[nbor].A) + (tauM['nborM'] * M[k-1, nbor])

                Mi = np.random.binomial(1, expit(sumM))

                M[k,i] = Mi

        return M

In [5]:
#Function to perform an intervention in the network
#Inputs: nodeList - dictionary representation of Units in graph, aVal - interventional value to set for A for ALL units in network
#Output: nodeListIntervention - modified nodeList post-intervention (but not accounting for downstream effects)
def doIntervention(nodeList, aVal):
    nodeListIntervention = copy.deepcopy(nodeList)
    for node in nodeListIntervention:
        nodeListIntervention[node].A = aVal
    return nodeListIntervention

In [6]:
%%time
Y_GT = 0
#perform ground truth calculations 5 times and calculate average effect to reduce effect of
#differences between Gibbs sampler runs for calculating M's
for iterations in range(5):
    print(iterations)
    Y_iter = 0
    #generate data for C, U as in generateData.py
    for node in graph.adj:
        nodeList[node] = unit(np.zeros(3), np.zeros(3), graph.adj[node])
        for j in range(len(Cscale)):
            nodeList[node].C[j] = np.random.beta(Cscale[j][0], Cscale[j][1])
            nodeList[node].U[j] = np.random.beta(Uscale[j][0], Uscale[j][1])
    
    #Perform interventions for A <- 1 and A <- 0
    A1_NetEffect = doIntervention(nodeList, 1)
    A0_NetEffect = doIntervention(nodeList, 0)

    #Perform Gibbs sampling under intervention to get samples of M's
    initMat1 = np.random.binomial(1, .5, (1051, len(nodeList)))
    initMat0 = np.random.binomial(1, .5, (1051, len(nodeList)))
    NetEffect1 = Gibbs(1050, initMat1, tauM, A1_NetEffect)
    NetEffect0 = Gibbs(1050, initMat0, tauM, A0_NetEffect)

    NetEffect1 = NetEffect1[1001:]
    NetEffect1 = NetEffect1[::10,]
    NetEffect0 = NetEffect0[1001:]
    NetEffect0 = NetEffect0[::10,]
    
    #Extract 5 M samples from Gibbs and use these to calculate `Ground Truth' Y value including U as covariate
    for i in range(len(nodeList)):
        Yi = 0
        
        for j in range(5):
            sumY1 = tauY['intcp'] + (tauY['M'] * NetEffect1[j, i]) + (tauY['C0'] * nodeList[i].C[0])
            sumY1 += (tauY['C1'] * nodeList[i].C[1]) + (tauY['C2'] * nodeList[i].C[2]) + (tauY['U0'] * nodeList[i].U[0])
            sumY1 += (tauY['U1'] * nodeList[i].U[1]) + (tauY['U2'] * nodeList[i].U[2])
            for nbor in nodeList[i].adj:
                sumY1 += (tauY['nborA'] * A1_NetEffect[nbor].A)
            Y1 = np.random.binomial(1, expit(sumY1))

            sumY0 = tauY['intcp'] + (tauY['M'] * NetEffect0[j, i]) + (tauY['C0'] * nodeList[i].C[0])
            sumY0 += (tauY['C1'] * nodeList[i].C[1]) + (tauY['C2'] * nodeList[i].C[2]) + (tauY['U0'] * nodeList[i].U[0])
            sumY0 += (tauY['U1'] * nodeList[i].U[1]) + (tauY['U2'] * nodeList[i].U[2])
            for nbor in nodeList[i].adj:
                sumY0 += (tauY['nborA'] * A0_NetEffect[nbor].A)
            Y0 = np.random.binomial(1, expit(sumY0))
            
            Yi += Y1 - Y0
        
        #Average over 5 M samples
        Y_iter += Yi / 5
    
    #Average across all nodes in network
    Y_GT += Y_iter / len(nodeList)
    
#Average across 5 Gibbs runs
Y_GT = Y_GT / 5
print(Y_GT)

0
1
2
3
4
-0.4562400000000005
CPU times: user 7min 4s, sys: 1.28 s, total: 7min 5s
Wall time: 7min 8s


In [7]:
#store ground truth effect to disk
with open('./2000_3/groundTruth2000_3.pkl', 'wb') as fname:
    pickle.dump(Y_GT, fname)